# Imports
This the script for prepossing the data to create data ready for the TDA mapper algorithm

In [11]:
import sys
import pandas as pd
from geopy import distance
import math
pd.set_option('display.max_columns', None)
import numpy as np
print("imports done")

imports done


# File
From the resulting `R` script written by Amber and `python` script written by Alaina, grab the cleaned data file.

In [12]:
filePath = r"C:\Users\forre\Desktop\REU\TDA\github\UMR-TDA-2021\pools EDA\pool data\cleaned_data.csv"
dataFrame = pd.read_csv(filePath, low_memory = False)
print("dataFrame Made")

dataFrame Made


# Filter for your pool



In [13]:
dataFrame = dataFrame[dataFrame['FLDNUM'] == 4]
print(dataFrame.shape)
dataFrame  = dataFrame.reset_index(drop = True)
dataFrame.head()

(14963, 18)


,SHEETBAR,TN,TP,TEMP,DO,TURB,COND,VEL,SS,WDP,CHLcal,SECCHI,LONGITUDE,LATITUDE,DATE,FLDNUM,LOCATCD,STRATUM
0,44000527,-0.200,0.082,3.9,10.4,32.0,223.0,0.0,NaN,1.10,NaN,40.0,-90.664712,38.920056,12/22/1993,4,M235.5D,NaN
1,44000719,1.834,-0.001,32.1,11.8,50.0,424.0,0.0,122.8,0.83,NaN,19.0,-90.233998,38.900486,07/06/1994,4,M206.0S,NaN
2,44001941,-0.200,-0.001,22.2,8.4,35.0,824.0,NaN,61.4,6.00,NaN,30.0,-90.140035,38.830625,09/20/1995,4,MO02.0X,NaN
3,44002201,NaN,NaN,2.4,14.2,15.0,455.0,NaN,-0.2,9.50,NaN,53.0,-90.683117,39.004773,11/29/1995,4,M241.4K,NaN
4,44002202,NaN,NaN,2.2,14.1,11.0,459.0,0.0,-2.0,0.94,NaN,64.0,-90.673138,38.940668,11/29/1995,4,M237.2G,NaN


# Functions
Here, we interpolate for missing data values. These occur when the ltrm data set has a missing value. The way it is is computed utilizes a $k$-nearest neighbors approach. A weighted average using the $k$ nearest points is used to compute the missing value. The current algorithm does not occur for temporal differences, however. To resolve this, data can be inputed one year at a time (or another time interval), and that way only data close in the point of time will be considered.


In [19]:
def predict(df, hashtable, naVar, k):
    df["PREDICTED_" + naVar] = df[naVar]
    naIndices = np.argwhere(np.isnan(np.array(df[naVar]))).flatten()
    print("For " + naVar + " we will interpolate " + str(len(naIndices)) + " points.")
    for index in naIndices:
        distances, neighbors = k_nearest_neighbors(df, index, naVar, hashtable, k)
        df.loc[index, "PREDICTED_" + naVar] = interpolate(df, distances, neighbors, naVar)
    print(naVar + " interpolation success")

def transform(minimum, maximum, x):
    return (1 / (maximum - minimum) ) * (x - minimum)

def dist(point1, point2):
    return distance.distance(point1, point2).km
    
def construct_hashtable(df):
    #get hashtable information
    data_length = math.sqrt(df.shape[0])
    #print("data_length: " + str(data_length))
    interval_length = 1 / data_length
    lat_minimum = df[["LATITUDE"]].min()[0] - 1
    lat_maximum = df[["LATITUDE"]].max()[0] + 1
    long_minimum = df[["LONGITUDE"]].min()[0] - 1
    long_maximum = df[["LONGITUDE"]].max()[0] + 1
    
    #construct hashtable
    hashtable = [[[] for x in range(int(data_length)+1)] for y in range(int(data_length)+1)]
    
    #populate hashtable
    for index, row in df.iterrows():
        r_lat = row['LATITUDE']
        r_long = row['LONGITUDE']
        lat = math.floor(transform(lat_minimum, lat_maximum, r_lat) / interval_length)
        long = math.floor(transform(long_minimum, long_maximum, r_long) / interval_length)
        #print("lat: " + str(lat))
        #print("long: " + str(long))
        hashtable[lat][long].append((index, r_lat, r_long))

    return hashtable

def k_nearest_neighbors(df, index, naVar, hashtable, k):

    distances = []
    neighbor_indices = []
    neighbors = {}
    
    data_length = math.sqrt(df.shape[0])
    interval_length = 1 / data_length
    lat_minimum = df[["LATITUDE"]].min()[0] - 1
    lat_maximum = df[["LATITUDE"]].max()[0] + 1
    long_minimum = df[["LONGITUDE"]].min()[0] - 1
    long_maximum = df[["LONGITUDE"]].max()[0] + 1
    
    
    row_na = df.loc[index]
    point_na = (row_na['LATITUDE'], row_na['LONGITUDE'])
    lat = math.floor(transform(lat_minimum, lat_maximum, point_na[0]) / interval_length)
    long = math.floor(transform(long_minimum, long_maximum, point_na[1]) / interval_length)

    for inx, latitude, longitude in hashtable[lat][long]:
        distance_km = dist(point_na, (latitude, longitude))
        if not np.isnan(df.loc[inx][naVar]) and distance_km != 0:
            distances.append(distance_km)
            if distance_km in neighbors.keys():
                neighbors[distance_km].append(inx)
            else:
                neighbors[distance_km] = [inx]
    
    if lat != 0:
        
        for inx, latitude, longitude in hashtable[lat - 1][long]:
                distance_km = dist(point_na, (latitude, longitude))
                if not np.isnan(df.loc[inx][naVar]) and distance_km != 0:
                    distances.append(distance_km)
                    if distance_km in neighbors.keys():
                        neighbors[distance_km].append(inx)
                    else:
                        neighbors[distance_km] = [inx]
                    
        if long != 0:
            for inx, latitude, longitude in hashtable[lat - 1][long - 1]:
                distance_km = dist(point_na, (latitude, longitude))
                if not np.isnan(df.loc[inx][naVar]) and distance_km != 0:
                    distances.append(distance_km)
                    if distance_km in neighbors.keys():
                        neighbors[distance_km].append(inx)
                    else:
                        neighbors[distance_km] = [inx]
        
        if long + 1 != len(hashtable):
            for inx, latitude, longitude in hashtable[lat - 1][long + 1]:
                distance_km = dist(point_na, (latitude, longitude))
                if not np.isnan(df.loc[inx][naVar]) and distance_km != 0:
                    distances.append(distance_km)
                    if distance_km in neighbors.keys():
                        neighbors[distance_km].append(inx)
                    else:
                        neighbors[distance_km] = [inx]
        
    if lat + 1 != len(hashtable):
        
        for inx, latitude, longitude in hashtable[lat + 1][long]:
                distance_km = dist(point_na, (latitude, longitude))
                if not np.isnan(df.loc[inx][naVar]) and distance_km != 0:
                    distances.append(distance_km)
                    if distance_km in neighbors.keys():
                        neighbors[distance_km].append(inx)
                    else:
                        neighbors[distance_km] = [inx]
                    
        if long != 0:
            for inx, latitude, longitude in hashtable[lat + 1][long - 1]:
                distance_km = dist(point_na, (latitude, longitude))
                if not np.isnan(df.loc[inx][naVar]) and distance_km != 0:
                    distances.append(distance_km)
                    if distance_km in neighbors.keys():
                        neighbors[distance_km].append(inx)
                    else:
                        neighbors[distance_km] = [inx]
        
        if long + 1 != len(hashtable):
            for inx, latitude, longitude in hashtable[lat + 1][long + 1]:
                distance_km = dist(point_na, (latitude, longitude))
                if not np.isnan(df.loc[inx][naVar]) and distance_km != 0:
                    distances.append(distance_km)
                    if distance_km in neighbors.keys():
                        neighbors[distance_km].append(inx)
                    else:
                        neighbors[distance_km] = [inx]
        
    if long != 0:
        for inx, latitude, longitude in hashtable[lat][long - 1]:
            distance_km = dist(point_na, (latitude, longitude))
            if not np.isnan(df.loc[inx][naVar]) and distance_km != 0:
                distances.append(distance_km)
                if distance_km in neighbors.keys():
                    neighbors[distance_km].append(inx)
                else:
                    neighbors[distance_km] = [inx]
        
    if long + 1 != len(hashtable):
        for inx, latitude, longitude in hashtable[lat][long + 1]:
            distance_km = dist(point_na, (latitude, longitude))
            if not np.isnan(df.loc[inx][naVar]) and distance_km != 0:
                distances.append(distance_km)
                if distance_km in neighbors.keys():
                    neighbors[distance_km].append(inx)
                else:
                    neighbors[distance_km] = [inx]
    
    distances.sort()
    distances = distances[0:k]
    for distance_km in distances:
        for inx in neighbors[distance_km]:
            neighbor_indices.append(inx)
    neighbor_indices = neighbor_indices[0:k]
    if len(neighbor_indices) < k and len(neighbor_indices) != 0:
        print("INTERPOLATING WITH " + str(len(neighbor_indices)) + " POINTS INSTEAD OF " + str(k) + " POINTS")
    if len(neighbor_indices) >= 2:
        return (distances, neighbor_indices)
    
    distances = []
    neighbors = {}
    neighbor_indices = []
    for inx, row in df.iterrows():
        distance_km = dist(point_na, (row['LATITUDE'], row['LONGITUDE']))
        if not np.isnan(df.loc[inx][naVar]) and distance_km != 0:
            distances.append(distance_km)
            if distance_km in neighbors.keys():
                    neighbors[distance_km].append(inx)
            else:
                neighbors[distance_km] = [inx]
    distances.sort()
    distances = distances[0:k]
    for distance_km in distances:
        for inx in neighbors[distance_km]:
            neighbor_indices.append(inx)
    neighbor_indices = neighbor_indices[0:k]
    return (distances, neighbor_indices)            
    

def interpolate(df, distances, neighbors, naVar):
    result = 0
    denominator = [1 / x for x in distances]
    denominator = sum(denominator)
    for i in range(len(distances)):
        result += ((1/distances[i]) / denominator) * df.loc[neighbors[i]][naVar]
    return result
print("Functions have been loaded")

Functions have been loaded


Here, we filter for the pool we are curious about, then run the interpolation code for the missing values. Copy the "predict" function for the new variables, and the number is the number of nearest neighbors that you want. When it is done, the data is displayed. To save the output, use pd.to_csv(path = ), and set the path to where you want the data frame to be saved.

In [ ]:
newData = dataFrame
newData["YEAR"] = pd.DatetimeIndex(dataFrame["DATE"]).year
years = newData["YEAR"].unique()
result = pd.DataFrame()
for year in years:
    currentSet = newData[(newData["YEAR"] == year)]
    currentSet = currentSet.reset_index(drop = True)
    #display(currentSet)
    hashTable = construct_hashtable(currentSet)
    predict(currentSet, hashTable, "TN", 2)
    predict(currentSet, hashTable, "TP", 2)
    print("Predicted for " + str(year))
    result.append(currentSet)
    

For TN we will interpolate 0 points.
TN interpolation success
For TP we will interpolate 0 points.
TP interpolation success
Predicted for 1993
For TN we will interpolate 0 points.
TN interpolation success
For TP we will interpolate 0 points.
TP interpolation success
Predicted for 1994
For TN we will interpolate 2 points.
INTERPOLATING WITH 1 POINTS INSTEAD OF 2 POINTS
INTERPOLATING WITH 1 POINTS INSTEAD OF 2 POINTS
TN interpolation success
For TP we will interpolate 2 points.
INTERPOLATING WITH 1 POINTS INSTEAD OF 2 POINTS
INTERPOLATING WITH 1 POINTS INSTEAD OF 2 POINTS
TP interpolation success
Predicted for 1995
For TN we will interpolate 0 points.
TN interpolation success
For TP we will interpolate 180 points.
TP interpolation success
Predicted for 1996
For TN we will interpolate 200 points.
TN interpolation success
For TP we will interpolate 200 points.
TP interpolation success
Predicted for 1997
For TN we will interpolate 204 points.
TN interpolation success
For TP we will interpol

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(result)